Import data and modules
-----

In [1]:
import pandas as pd
import numpy as np
from time import time
#import matplotlib.pyplot as plt
#%matplotlib inline

# load the dataset
list_ = []
for file_ in ['Database2014.txt']:
    #,'Database2013.txt']:#,'Database2012.txt','Database2011.txt',]:
    print('Loading', file_, '...')
    try:
        #df = pd.read_csv(file_,index_col=None, header=0)
        df = pd.read_csv(file_, engine = 'python', sep='\t', index_col='Obs.')
    except OSError:
        print('!!file not found!!')
        break
    list_.append(df)
data = pd.concat(list_)

for key in data.keys():
    if key.startswith('Unnamed'):
        data = data.drop(key, 1)
print('Number of rows in dataset:',len(data))

Loading Database2014.txt ...
Number of rows in dataset: 72888


In [2]:
# Select interesting stuff
# --------------

# Load Code manual
import pickle
code_manual = pickle.load( open( "CodeManual.p", "rb" ) )

# Select interesting features and map them to their respective
#   names in the code manual
translate_column_names = {
'driverdrowsy' : 'Drowsy driver',
'ptype' : 'Person Type',
'druginv' : 'Police Reported Drug Involvement',
'heavytruck': 'Large Truck Related',
'schlbus' : 'School Bus Related',
'sex' : 'Sex',
'race' : 'Race',
'reljuncinter' : 'Relation To Junction: Within Interchange Area',
'atmcond': 'Atmospheric Condition (1)',
'holiday' : 'Holiday Related',
'nhs' : 'National Highway System',
'hispanic' : 'Hispanic',
'rfun' : 'Roadway Function Class',
'lightcond' : 'Light Condition',
'speeding' : 'Speeding',
'dayofweek' : 'Day Of Week',
}

also_interesting = ['age', 'alcres']

labelsOfInter = list(translate_column_names.keys())
labelsOfInter.extend(also_interesting)
for item in['heavytruck','schlbus','reljuncinter']:
    labelsOfInter.remove(item)


data = data[labelsOfInter].applymap(lambda x: -1 if x == '.' else int(x))

In [3]:
labelsOfInter

['nhs',
 'holiday',
 'speeding',
 'race',
 'rfun',
 'sex',
 'dayofweek',
 'atmcond',
 'hispanic',
 'druginv',
 'driverdrowsy',
 'ptype',
 'lightcond',
 'age',
 'alcres']

Reduce dimensionality
-----------

In [4]:
# Collapse some features values
# ------------

# Categorize on number of Fatalities
def NumFatalitiesToCategory(num):
    if num <3:
        return 1
    elif (num>=3) and (num<6):
        return 2
    return 3

# Reduce categories in 'Light Condition'
code_manual[translate_column_names['lightcond']][11] = 'Dawn/Dusk'
code_manual[translate_column_names['lightcond']][12] = 'Dark'
def Collaps_lightcond(num):
    if num == 4 or num == 5:
        return 11
    elif num in [6,8,9]: #unknown
        return -1
    return num

# Reduce categories in "Roadway Function Class"
code_manual[translate_column_names['rfun']][21] = 'Rural'
code_manual[translate_column_names['rfun']][22] = 'Urban'
def Collaps_rfun(num):
    if num in range(1,10):
        return 21
    elif num in range(11,20):
        return 22
    elif num == 99: #unknown
        return -1
    return num


# Reduce categories in "Holiday Related"
code_manual[translate_column_names['holiday']][-1] = 'Not Holiday or Unknow date'
code_manual[translate_column_names['holiday']][1] = 'Was Holiday'
def Collaps_holiday(num):
    if num >0:
        return 1
    return num


# Reduce categories in "Atmospheric Condition (1)"
code_manual[translate_column_names['atmcond']][1] = 'Clear or Cloudy'
code_manual[translate_column_names['atmcond']][2] = 'Precipitation'
code_manual[translate_column_names['atmcond']][4] = 'Snow'
code_manual[translate_column_names['atmcond']][6] = "Severe \
Crosswinds, Blowing Sand, Soil, Dirt",
def Collaps_atmcond(num):
    if num in [8, 0, 98, 99]: #other or not known
        return -1
    elif num in [1,10]:
        return 1
    elif num in [2,12,3]:
        return 2
    elif num in [6,7]:
        return 6
    return num


# Reduce categories in "Person Type"
code_manual[translate_column_names['ptype']][1] = 'In a Motor vehicle'
code_manual[translate_column_names['ptype']][4] = 'Non-Motor vehicle'
def Collaps_ptype(num):
    if num in [-1,19]: #other or not known
        return -1
    elif num in [1,2,9,3]:
        return 1
    elif num in [4,8,6,7]:
        return 4
    return num

# Reduce categories in "Hispanic"
code_manual[translate_column_names['hispanic']][10] = 'Is Hispanic'
def Collaps_hispanic(num):
    if num in [-1,99,0]: #other or not known
        return -1
    elif num in range(1,7):
        return 10
    return num


# Reduce categories in "Race"
code_manual[translate_column_names['race']][30] = 'American Indian or Hawaiian'
code_manual[translate_column_names['race']][31] = 'Asian (not Indian)'
code_manual[translate_column_names['race']][32] = 'Indian'
def Collaps_race(num):
    if num in [-1, 0, 98, 99, 97]: #other or not known
        return -1
    elif num in [3,6]:  
        return 30
    elif num in [4,5,7,28,38,48,58,68,78]:  
        return 31
    elif num in [18,19]:  
        return 32
    return num


In [5]:
# apply reduction of categories cardinality
data.lightcond = data.lightcond.apply(Collaps_lightcond )
data.rfun      = data.rfun.apply(Collaps_rfun  )
data.holiday   = data.holiday.apply(Collaps_holiday )
data.atmcond   = data.atmcond.apply(Collaps_atmcond )
data.ptype     = data.ptype.apply(Collaps_ptype )
data.hispanic  = data.hispanic.apply(Collaps_hispanic )
data.race      = data.race.apply(Collaps_race)

In [6]:
# Map different types of 'unknown' to a single number
unknown = {'druginv': [8,9],
           'nhs'    : [9],
           'reljuncinter': [8,9],
           'dayofweek': [-1,9],
           'sex'     : [8,9],
           'age'     : [-1, 998, 999],
           'alcres'  : [95, 96, 97, 98, 99],
           'druginv' : [-1, 8, 9]
          }

# collaps all keys for 'unknown' into '-1' value
for feature in unknown.keys():
    try:
        data[feature] = data[feature].apply(lambda x: 
                                            x if x not in unknown[feature] 
                                            else -1)
    except KeyError:
        pass

One-Hot encoder where needed
-------------

In [7]:
def categorize_a_label(dataset, labl_to_categ = 'drug'):
    '''
    Use One-hot encoder on a categorical feature,
    join it back to the original dataframe
    and return the new dataframe
    '''
    tt = dataset[labl_to_categ]
    tt_df = pd.get_dummies(tt)
    tt_df.columns = [ '%s:%d'%(labl_to_categ, value) for value in tt_df.keys() ]
    dataset = dataset.join(tt_df)
    dataset = dataset.drop(labl_to_categ,1)
    return dataset

categorical_features_list = ['lightcond', 'schlbus', 'druginv', 'nhs', 
                             'speeding', 'rfun', 'holiday',
                           'atmcond', 'heavytruck', 'reljuncinter', 
                             'ptype', 'dayofweek',
                           'hispanic', 'race', 'driverdrowsy']
non_categ_features_list = [ 'sex', 'age', 'alcres' ]

target_name = 'dayofweek'

this on

In [8]:
for feature in data.keys():
    if feature == target_name:
        continue
    elif feature in categorical_features_list:
        data = categorize_a_label(data, labl_to_categ = feature)
    elif feature in non_categ_features_list:
        continue
    else:
        print("!!! Don't know whether", feature, 'is categorical or not')
        break

Select n features, analyze and return score
-----

In [9]:
max_depth_for_classifier = 5

In [10]:
def run_analysis(clean_dataset, target_name, returnClassifier=False):
    # Separate into train_features, train_label and test_feature, test_label
    threshold = int(round(len(clean_dataset)*.70))
    train_features  = clean_dataset[:threshold].drop(target_name,1)
    test_feature    = clean_dataset[threshold:].drop(target_name,1)
    train_label     = clean_dataset[target_name][:threshold]
    test_label      = clean_dataset[target_name][threshold:]
    
    # Train decision tree
    from sklearn.tree import DecisionTreeClassifier
    clf = DecisionTreeClassifier(max_depth = max_depth_for_classifier)
    clf.fit(train_features, train_label)
    predict = clf.predict(test_feature)
     
    if returnClassifier:
        return clf.score(test_feature, test_label), clf
    return clf.score(test_feature, test_label)

In [50]:
from itertools import combinations as comb
output = {}
target_name = 'dayofweek'

In [12]:
ww0 = ''
ww = ''
for sel_feat in comb(labelsOfInter, 4):
    features = list(sel_feat)
   
    if target_name in features:
        features = features.remove(target_name)
        continue
    if str(features) in output.keys():
        continue
        
    #is it a new word?
    if features[0] != ww:
        print('\n--', features[0])
        ww = features[0]
    try:
        if features[1] != ww0:
            print(features[1], end=", ")
            ww0 = features[1]      
    except IndexError:
        pass
    
    # There are many keys in the dataset for one feature
    # because we passed the one-hot encoder.
    #  we want to select all keys corresponding to
    #  the given feature
    sel_keys = []
    for feat in features:
        for key in data.keys():
            if key.startswith(feat):
                sel_keys.append(key)
    sel_keys.append(target_name)
    clean = data[sel_keys]#.copy()

    output[str(features)] = run_analysis(clean, target_name)


-- nhs
holiday, speeding, race, rfun, sex, atmcond, hispanic, druginv, driverdrowsy, ptype, lightcond, 
-- holiday
speeding, race, rfun, sex, atmcond, hispanic, druginv, driverdrowsy, ptype, lightcond, 
-- speeding
race, rfun, sex, atmcond, hispanic, druginv, driverdrowsy, ptype, lightcond, 
-- race
rfun, sex, atmcond, hispanic, druginv, driverdrowsy, ptype, lightcond, 
-- rfun
sex, atmcond, hispanic, druginv, driverdrowsy, ptype, lightcond, 
-- sex
atmcond, hispanic, druginv, driverdrowsy, ptype, lightcond, 
-- atmcond
hispanic, druginv, driverdrowsy, ptype, lightcond, 
-- hispanic
druginv, driverdrowsy, ptype, lightcond, 
-- druginv
driverdrowsy, ptype, lightcond, 
-- driverdrowsy
ptype, lightcond, 
-- ptype


In [13]:
pp = pd.Series(output)
pp =pp.sort_values(ascending=False)[:10]

#plot the top 10
pl = pp.to_frame('score')
pl = pl.reset_index()
pl.columns = ['feat', 'score']
pl['rel_score'] = pl.score.apply(lambda x: x/max(pl.score)) 
pl

,feat,score,rel_score
0,"['nhs', 'race', 'druginv', 'alcres']",0.194869,1.000000
1,"['nhs', 'rfun', 'ptype', 'alcres']",0.194869,1.000000
2,"['rfun', 'driverdrowsy', 'ptype', 'alcres']",0.194823,0.999765
3,"['holiday', 'rfun', 'ptype', 'alcres']",0.194732,0.999296
4,"['race', 'hispanic', 'druginv', 'alcres']",0.194640,0.998827
5,"['rfun', 'sex', 'hispanic', 'alcres']",0.194320,0.997184
6,"['holiday', 'driverdrowsy', 'ptype', 'alcres']",0.194320,0.997184
7,"['holiday', 'rfun', 'hispanic', 'alcres']",0.194228,0.996714
8,"['rfun', 'hispanic', 'driverdrowsy', 'alcres']",0.194228,0.996714
9,"['race', 'ptype', 'age', 'alcres']",0.194183,0.996480


Plot top results
-------

In [14]:
from bokeh.charts import Bar
from bokeh.io import show, output_notebook
from bokeh.models import Range1d
from bokeh.charts.attributes import CatAttr
from bokeh.plotting import show, output_file
#output_notebook()
output_file("barchart.html", title="results")

#TOOLS = "pan,wheel_zoom,box_zoom,reset,resize,save"
#fig = figure(tools=TOOLS, toolbar_location="left", logo="grey", plot_width=120)


p = Bar(pl, label=CatAttr(columns=['feat'], sort=False),
        values='rel_score', 
        title="Scores per set of features",
        color="blue",
        ylabel="score (relative to max score)",
        xlabel='',
        plot_width = 900)
p.y_range = Range1d(round(pl.rel_score.iloc[-1],3),pl.rel_score.iloc[1])
show(p)


#Bar(pl).io.show()

Export the tree
---------

In [15]:
def rules(clf, features, labels, node_index=0, simple_leaf_string=True):
    """Structure of rules in a fit decision tree classifier

    Parameters
    ----------
    clf : DecisionTreeClassifier
        A tree that has already been fit.

    features, labels : lists of str
        The names of the features and labels, respectively.

    """
    node = {}
    if clf.tree_.children_left[node_index] == -1:  # indicates leaf
        count_labels = zip(clf.tree_.value[node_index, 0], labels)
        node['name'] = ', '.join(('{} of {}'.format(int(count), label)
                                  for count, label in count_labels))
        if simple_leaf_string:
            node['name'] = simplify_string(node['name'])
    else:
        feature = features[clf.tree_.feature[node_index]]
        threshold = clf.tree_.threshold[node_index]
        node['name'] = '{} > {}'.format(feature, threshold)
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        node['children'] = [rules(clf, features, labels, right_index, simple_leaf_string),
                            rules(clf, features, labels, left_index, simple_leaf_string)]
    return node

def simplify_string(st):
    #st = '772 of Sun, 838 of Mon, 862 of Tue, 899 of Wed, 919 of Thu, 1024 of Fri, 994 of Sat'
    st = st.split(', ')
    nums = [ int(entry.split()[0]) for entry in st ]
    pred_day = label_dictionary[nums.index(max(nums))]
    return pred_day

In [58]:
# Get the tree for the winning set of features
#-------------

# retrieve winning set of features
ss = pl.feat.iloc[0]
for char in ['[', ']', "'"]:
    ss = ss.replace(char, '')
ff = ss.split(',')

sel_keys = []
for feat in ff:
    feat = feat.strip()
    for key in data.keys():
        if key.startswith(feat):
            sel_keys.append(key)
sel_keys.append(target_name)

# re-do analysis and catch classifier
clean = data[sel_keys]
num_, classfr = run_analysis(clean, target_name, returnClassifier=True)

In [62]:
if target_name in sel_keys:
    sel_keys.remove(target_name)

sel_keys

['nhs:-1',
 'nhs:0',
 'nhs:1',
 'race:-1',
 'race:1',
 'race:2',
 'race:30',
 'race:31',
 'race:32',
 'druginv:-1',
 'druginv:0',
 'druginv:1',
 'alcres']

In [63]:
import json
label_dictionary = 'Sun Mon Tue Wed Thu Fri Sat'.split()

r = rules(classfr, sel_keys, label_dictionary, simple_leaf_string=True)
with open('rules_maxdepth%d.json'%(max_depth_for_classifier), 'w') as f:
    f.write(json.dumps(r))